# Generate Polynomial to Construct Neurons 

In [1]:
from Poly_tensor_regressor import PolynomialTensorRegression
import numpy as np
from torch import nn
from seeds import random_seed
from neurons_better import Neurons
import torch
from mydata import MyData
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from ny_torch import ny2tensor


## California Housing Data

In [4]:
ori_data = datasets.fetch_california_housing()  # 加州房价数据——(20640, 8)
x_data = ori_data.data
y_data = ori_data.target.reshape(-1, 1)
decomp_rank = 3
poly_order = 4

neuron = PolynomialTensorRegression(decomp_rank, 
                                    poly_order, 
                                    method='cp', 
                                    reg_lambda_w=0.01, 
                                    reg_lambda_c=0.01) 

neuron.fit(x_data,y_data)
print(neuron.neuron)

- 2.1893 + 0.6835 @ x**1 - 0.4742 @ x**2 + 0.6720 @ x**3


In [8]:
def main(seed):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ori_data = datasets.fetch_california_housing()  # 加州房价数据——(20640, 8)
    x_data = ori_data.data
    y_data = ori_data.target.reshape(-1, 1)

    train_x, x_test, train_y, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=seed)
    x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.15, random_state=seed)

    scaler_x = MinMaxScaler(feature_range=(-1, 1))
    x_train = ny2tensor(scaler_x.fit_transform(x_train))
    x_valid = ny2tensor(scaler_x.transform(x_valid))
    x_test = ny2tensor(scaler_x.transform(x_test))

    scaler_y = MinMaxScaler(feature_range=(-1, 1))
    y_train = ny2tensor(scaler_y.fit_transform(y_train))
    y_valid = ny2tensor(scaler_y.transform(y_valid))
    y_test = ny2tensor(scaler_y.transform(y_test))

    max_epoch = 300
    batch_size = 128

    trainset = MyData(x_train, y_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    # =======================================================================================================

    random_seed(seed)
    sr1 = '0.06@x**3 + 0.14@x + 0.76'

    net1 = nn.Sequential(Neurons(8, 6, sr1),
                         nn.Sigmoid(),

                         Neurons(6, 3, sr1),
                         nn.Sigmoid(),

                         Neurons(3, 1, sr1),
                         ).to(device)

    cost1 = nn.MSELoss().to(device)
    optimizer1 = torch.optim.RMSprop(net1.parameters(), lr=0.001)

    initial_score1 = float("inf")

    for k in range(max_epoch):
        for inputs1, labels1 in trainloader:
            inputs1 = inputs1.to(device)
            labels1 = labels1.to(device)
            predict1 = net1(inputs1)
            loss1 = cost1(predict1, labels1)
            optimizer1.zero_grad()
            loss1.backward()
            optimizer1.step()

        # validation
        with torch.no_grad():
            error1 = mean_squared_error(net1(x_valid.to(device)).cpu(), y_valid)
            if error1 < initial_score1:
                initial_score1 = error1
                model_best1 = net1

        if (k+1) % 100 == 0:
            print('epoch: ', k + 1, ' net_1_val_error: ', error1)

    # prediction
    with torch.no_grad():
        test_error1 = mean_squared_error(model_best1(x_test.to(device)).cpu(), y_test)

    print('net_1_test_error: ', test_error1, '\n')

# =================================================================================================
# =================================================================================================

    random_seed(seed)
    sr2 = '- 2.1893 + 0.6835 @ x**1 - 0.4742 @ x**2 + 0.6720 @ x**3'

    net2 = nn.Sequential(Neurons(8, 6, sr2),
                         nn.Sigmoid(),

                         Neurons(6, 3, sr2),
                         nn.Sigmoid(),

                         Neurons(3, 1, sr2),
                         ).to(device)

    cost2 = nn.MSELoss().to(device)
    optimizer2 = torch.optim.RMSprop(net2.parameters(), lr=0.001)

    initial_score2 = float("inf")

    for k in range(max_epoch):
        for inputs2, labels2 in trainloader:
            inputs2 = inputs2.to(device)
            labels2 = labels2.to(device)
            predict2 = net2(inputs2)
            loss2 = cost2(predict2, labels2)
            optimizer2.zero_grad()
            loss2.backward()
            optimizer2.step()

        # validation
        with torch.no_grad():
            error2 = mean_squared_error(net2(x_valid.to(device)).cpu(), y_valid)
            if error2 < initial_score2:
                initial_score2 = error2
                model_best2 = net2

        if (k+1) % 100 == 0:
            print('epoch: ', k + 1, ' net_2_val_error: ', error2)

    # prediction
    with torch.no_grad():
        test_error2 = mean_squared_error(model_best2(x_test.to(device)).cpu(), y_test)

    print('net_2_test_error: ', test_error2, '\n')

    return test_error1, test_error2


if __name__ == '__main__':

    error1_list = []
    error2_list = []
    sv = np.empty((1, 4))

    for i in range(10):
        print('run times:', i + 1)
        error_1, error_2 = main(i + 1)

        error1_list.append(error_1)
        error2_list.append(error_2)

    sv[0, 0] = np.mean(error1_list)
    sv[0, 1] = np.std(error1_list)

    sv[0, 2] = np.mean(error2_list)
    sv[0, 3] = np.std(error2_list)

    np.savetxt('california_housing_' + 'result.csv', sv, delimiter=',')


run times: 1
epoch:  100  net_1_val_error:  0.07589223
epoch:  200  net_1_val_error:  0.07318521
epoch:  300  net_1_val_error:  0.07165239
net_1_test_error:  0.07525507 

epoch:  100  net_2_val_error:  0.07415062
epoch:  200  net_2_val_error:  0.07245491
epoch:  300  net_2_val_error:  0.07019166
net_2_test_error:  0.07178815 

run times: 2
epoch:  100  net_1_val_error:  0.07690656
epoch:  200  net_1_val_error:  0.07268754
epoch:  300  net_1_val_error:  0.069136046
net_1_test_error:  0.07420971 

epoch:  100  net_2_val_error:  0.07141983
epoch:  200  net_2_val_error:  0.06734463
epoch:  300  net_2_val_error:  0.06568623
net_2_test_error:  0.07058753 

run times: 3
epoch:  100  net_1_val_error:  0.080942765
epoch:  200  net_1_val_error:  0.07413984
epoch:  300  net_1_val_error:  0.07329793
net_1_test_error:  0.07251012 

epoch:  100  net_2_val_error:  0.07620826
epoch:  200  net_2_val_error:  0.07153936
epoch:  300  net_2_val_error:  0.07115678
net_2_test_error:  0.06974656 

run times: 

In [11]:
ori_data = datasets.fetch_openml('house_sales', version = 8)  # 房价数据——(21613, 15)
x_data = np.array(ori_data.data, dtype=np.float32)
y_data = np.array(ori_data.target, dtype=np.float32).reshape((-1, 1))
decomp_rank = 3
poly_order = 4

neuron = PolynomialTensorRegression(decomp_rank, 
                                    poly_order, 
                                    method='cp', 
                                    reg_lambda_w=0.01, 
                                    reg_lambda_c=0.01) 

neuron.fit(x_data,y_data) 
print(neuron.neuron)